In [3]:
# see Leo's analysis here: https://github.com/saezlab/best_practices_ParTIpy/tree/main
from pathlib import Path

import plotnine as pn
import scanpy as sc
import partipy as pt

import harmonypy as hm


figure_dir = Path("figures") / "fibroblast_cross_condition"
figure_dir.mkdir(exist_ok=True, parents=True)
sc.settings.figdir = figure_dir

output_dir = Path("output") / "fibroblast_cross_condition"
output_dir.mkdir(exist_ok=True, parents=True)

adata = pt.read_h5ad("/home/pschaefer/fibroblast_cross_condition_partipy.h5ad")